In [3]:
%load_ext autoreload
%autoreload 2

In [100]:
# Import the usual suspects
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import catboost
import pandas as pd

import sys, os
sys.path.insert(0, os.path.abspath('..'))
#from runremotely.runremote import Compute, runremotely
from runremotely.server import ec2, requirements as reqs
from runremotely.runremote import runremotely

In [101]:
data = pd.read_csv('data.csv')

y = np.log(data.SalePrice)
X = data.drop(['SalePrice', 'Id'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
                          X, y, random_state=42, test_size=.33)

print(X_train.shape)

(978, 246)


In [154]:
param_grid = {'learning_rate': np.arange(0.05, 0.5, 0.01),
    'depth': range(1,3,1),
    'l2_leaf_reg': np.arange(0.1,2.0,0.1)}

model = CatBoostRegressor()
#grid = model.grid_search(param_grid, X_train, y=y_train, cv=5,plot=False)

In [157]:
@runremotely(globals(), instance="c5.9xlarge")
def execute():
    grid = model.grid_search(param_grid, X_train, y=y_train, cv=5,plot=False)

In [158]:
execute()

Requirements files: /var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmplacudpn2.reqs
Creating key pair [/var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmpc9whcu_5.tp]
Launching instance (c5.9xlarge)
EC2 instance created:
	Id: i-015c56cbb0856cd19
	Public IP address: 52.16.16.129
Remote connection failed [[Errno None] Unable to connect to port 22 on 52.16.16.129]. 
Trying again in 20secs...
Transfering [/var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmpbe10np6q.tp] to ec2
Transfering [../template.py] to ec2
Transfering [/var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmplacudpn2.reqs] to ec2
Executing [sudo yum -y install python37]
Executing [curl -O https://bootstrap.pypa.io/get-pip.py]
Error:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1824k  100 1824k    0     0  31.2M      0 --:--:-- --:--:-- --:--:-- 30.7M

Executing [sudo  python3 get-pip.py --user]
Error:   W

Retrieving [result.pickle] from ec2

Deleting remote key pair...
Terminating EC2 instance [i-015c56cbb0856cd19]
Deleting local key pair file
Done


FileNotFoundError: [Errno 2] No such file

In [35]:
from pipreqs import pipreqs as pi
from pipreqsnb import pipreqsnb as pnb
import ast

In [36]:
def get_all_imports(contents):
    imports = set()
    raw_imports = set()
    tree = ast.parse(contents)
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for subnode in node.names:
                raw_imports.add(subnode.name)
        elif isinstance(node, ast.ImportFrom):
            raw_imports.add(node.module)
    
    for name in [n for n in raw_imports if n]:
        cleaned_name, _, _ = name.partition('.')
        imports.add(cleaned_name)
    
    return imports# - (set(candidates) & imports)    

In [70]:
import types

def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

excludes = ['builtins', 'types', 'sys']

imported_modules = [module for module in imports() if module not in excludes]

clean_modules = []

for module in imported_modules:

    sep = '.'  # to handle 'matplotlib.pyplot' cases
    rest = module.split(sep, 1)[0]
    clean_modules.append(rest)

changed_imported_modules = list(set(clean_modules))  # drop duplicates

pip_modules = !pip freeze  # you could also use `!conda list` with anaconda

for module in pip_modules:
    name, version = module.split('==')
    if name in changed_imported_modules:
        print(name + '\t' + version)

catboost	0.23
numpy	1.18.1
pandas	1.0.3
pipreqs	0.4.10
pipreqsnb	0.1.3


In [81]:
import pkg_resources
import types
def get_imports():
    imports = []
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]
        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
        imports.append(name)
    return list(set(imports))

imports = get_imports()

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

pipreqsnb==0.1.3
pipreqs==0.4.10
pandas==1.0.3
numpy==1.18.1
catboost==0.23


In [ ]:
json
numpy
catboost
pandas
sys
os
runremotely.server.ec2
types
importlib_metadata
pipreqs.pipreqs
pipreqsnb.pipreqsnb
ast

In [67]:
def get_imports():
    imports = []
    for name, val in globals().items():
        if 'ih' in name:
        #if val not in [None, types.ModuleType]:
            #if isinstance(val, types.ModuleType) and name not in ['__builtin__', '__builtins__','runremote']:
            #print(val)
                #imports.append(val.__name__)
            #print(val[1])
            valid_lines = pnb.clean_invalid_lines_from_list_of_lines(val)
            source = ''.join(valid_lines)
            imports += pnb.get_import_string_from_source(source)
    return imports
get_imports()

SyntaxError: invalid syntax (<unknown>, line 20)

In [37]:
nb_imports = []
nb_file = "test.ipynb"
nb = json.load(open(nb_file, 'r'))
for cell in nb['cells']:
    if cell['cell_type'] == 'code':
        valid_lines = pnb.clean_invalid_lines_from_list_of_lines(cell['source'])
        source = ''.join(valid_lines)
        nb_imports += pnb.get_import_string_from_source(source)
nb_imports

['from sklearn.model_selection import train_test_split',
 'from catboost import CatBoostRegressor',
 'import catboost',
 'import pandas as pd',
 'import sys, os',
 'from runremotely.server import ec2',
 'from runremotely import runremote',
 'import types',
 'import importlib_metadata',
 'import pip #needed to use the pip functions',
 'import pkg_resources',
 'import types',
 'import importlib_metadata',
 'from pipreqs import pipreqs as pi',
 'from pipreqsnb import pipreqsnb as pnb',
 'import ast']

In [46]:
nb_imports

['json',
 'numpy',
 'catboost',
 'pandas',
 'sys',
 'os',
 'runremotely.server.ec2',
 'types',
 'importlib_metadata',
 'pipreqs.pipreqs',
 'pipreqsnb.pipreqsnb',
 'ast']

In [44]:
allimports = get_all_imports('\n'.join(nb_imports))
allimports

set()

In [ ]:
candidates = pi.get_pkg_names(allimports)
candidates

In [ ]:
pypi_server = "https://pypi.python.org/pypi/"
proxy = None

local = pi.get_import_local(candidates)
difference = [x for x in candidates if x.lower() not in [z['name'].lower() for z in local]]
imports = local + pi.get_imports_info(difference,
                   proxy=proxy,
                   pypi_server=pypi_server)
imports